# Creating Word Vectors with word2vec

Let's start with NLTK

#### Load Dependencies

In [ ]:
import nltk

from nltk.tokenize import word_tokenize, sent_tokenize

import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE

import pandas as pd

from bokeh.io import output_notebook
from bokeh.plotting import show, figure

%matplotlib inline

In [ ]:
nltk.download('punkt')

#### Load Data

In [ ]:
nltk.download('gutenberg')

In [ ]:
from nltk.corpus import gutenberg

In [ ]:
gutenberg.fileids()

#### Tokenize Text

In [ ]:
# Due to lack of resources, I'm not working with the full Gutenberg dataset (18 books).
gberg_sent_tokens = sent_tokenize(gutenberg.raw(fileids=['austen-sense.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt']))

In [ ]:
gberg_sent_tokens[0:5]

In [ ]:
gberg_sent_tokens[1]

In [ ]:
word_tokenize(gberg_sent_tokens[1])

In [ ]:
word_tokenize(gberg_sent_tokens[1])[14]

In [ ]:
# Due to lack of resources, I'm not working with the full Gutenberg dataset (18 books).
gberg_sents = gutenberg.sents(fileids=['austen-sense.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt'])

In [ ]:
gberg_sents[0:5]

In [ ]:
gberg_sents[4][14]

In [ ]:
gutenberg.words()

In [ ]:
# Due to lack of resources, I'm not working with the full Gutenberg dataset (18 books).
len(gutenberg.words(fileids=['austen-sense.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt']))

#### Run Word2Vec

In [ ]:
# size == dimensions
# window 10: 20 context words, 10 to the left and 10 to the right
model = Word2Vec(sentences=gberg_sents, size=64, sg=1, window=10, min_count=5, seed=42, workers=2)

In [ ]:
# We don't have to save the model if we don't want to. It's being done here as demonstration.
model.save('raw_gutenberg_model.w2v')

#### Explore the Model

In [ ]:
model = Word2Vec.load('raw_gutenberg_model.w2v')

In [ ]:
model['house']

In [ ]:
len(model['house'])

In [ ]:
model.most_similar('house')

In [ ]:
model.most_similar('think')

In [ ]:
model.most_similar('day')

In [ ]:
model.most_similar('father')

In [ ]:
model.doesnt_match('mother father daughter house'.split())

In [ ]:
model.similarity('father', 'mother')

In [ ]:
model.most_similar(positive=['father', 'woman'], negative=['man'])

In [ ]:
model.most_similar(positive=['son', 'woman'], negative=['man'])

In [ ]:
model.most_similar(positive=['husband', 'woman'], negative=['man'])

In [ ]:
model.most_similar(positive=['king', 'woman'], negative=['man'], topn=50)

#### Reduce word vector dimensionality with t-SNE

t-Distributed Stochastic Name Embedding

In [ ]:
len(model.wv.vocab)

In [ ]:
X = model[model.wv.vocab]

In [ ]:
tsne = TSNE(n_components=3, n_iter=250)

In [ ]:
X_2d = tsne.fit_transform(X)

In [ ]:
coords_df = pd.DataFrame(X_2d, columns=['x', 'y', 'z'])
coords_df['token'] = model.wv.vocab.keys()

In [ ]:
coords_df.head()

In [ ]:
coords_df.to_csv('raw_gutenberg_tsne.csv', index=False)

#### Visualise 2D representation of word vectors

In [ ]:
coorrds_df = pd.read_csv('raw_gutenberg_tsne.csv')

In [ ]:
coords_df.head()

In [ ]:
_ = coords_df.plot.scatter('x', 'y', 'z', figsize=(8,8,8), marker='o', s=10, alpha=0.2)

In [ ]:
output_notebook()

In [ ]:
subset_df = coords_df.sample(n=1000)

In [ ]:
p = figure(plot_width=600, plot_height=600)
p.text(x=subset_df.x, y=subset_df.y, text=subset_df.token)

In [ ]:
show(p)